### Import Dependencies

In [1]:
import pandas as pd
import os
import numpy as np
from pprint import pprint
from sqlalchemy import create_engine, inspect
import psycopg2

## Process Stats Data

In [51]:
# Read from CSVs
reg_off_df = pd.read_csv('Data/nba_team_stats_data/nba_off.csv')
reg_def_df = pd.read_csv('Data/nba_team_stats_data/nba_def.csv')
reg_adv_df = pd.read_csv('Data/nba_team_stats_data/nba_adv_stats.csv')
post_off_df = pd.read_csv('Data/nba_team_stats_data/nba_post_off.csv')
post_def_df = pd.read_csv('Data/nba_team_stats_data/nba_post_def.csv')
post_adv_df = pd.read_csv('Data/nba_team_stats_data/nba_post_adv_stats.csv')

In [52]:
# Drop Unused Columns
reg_off_filtered_df = reg_off_df.drop(columns=['Rk'])
reg_def_filtered_df = reg_def_df.drop(columns=['Rk'])
reg_adv_filtered_df = reg_adv_df.drop(columns=['Rk', 'Unnamed: 17', 'Unnamed: 27', 'Arena', 'Attend.'])
post_off_filtered_df = post_off_df.drop(columns=['Rk'])
post_def_filtered_df = post_def_df.drop(columns=['Rk'])
post_adv_filtered_df = post_adv_df.drop(columns=['Rk', 'Unnamed: 15', 'Unnamed: 20'])

In [54]:
# Check column names if so desired

# reg_off_filtered_df.columns
# reg_def_filtered_df.columns
# reg_adv_filtered_df.columns
# post_off_filtered_df.columns
# post_def_filtered_df.columns
# post_adv_filtered_df.columns

### Process Regular Season and Post Season Aggregate Tables

In [62]:
regular_season_df = pd.read_csv('Data/nba_team_stats_data/nba_reg_compiled_stats.csv')
regular_season_df = regular_season_df.drop(['Unnamed: 0'],axis=1)
regular_season_df.columns = regular_season_df.columns.str.lower()

regular_season_df = regular_season_df.rename(columns = {'off_fg%':'off_fg_percent','off_3p%':'off_3p_percent','off_2p%':'off_2p_percent','off_ft%':'off_ft_percent',
                                     'def_fg%':'def_fg_percent','def_3p%':'def_3p_percent','def_2p%':'def_2p_percent',
                                    'def_ft%':'def_ft_percent','w/l%':'w_l_percent','ts%':'ts_percent','off_efg%':'off_efg_percent',
                                    'off_tov%':'off_tov_percent','off_orb%':'off_orb_percent','off_ft/fga':'off_ft_fga','def_efg%':'def_efg_percent',
                                    'def_tov%':'def_tov_percent','def_drb%':'def_drb_percent','def_ft/fga':'def_ft_fga', '3par':'three_par',
                                    'attend./g':'attend_g'})

# regular_season_df.columns

In [63]:
post_season_df = pd.read_csv('Data/nba_team_stats_data/nba_post_compiled_stats.csv')
post_season_df = post_season_df.drop(['Unnamed: 0'],axis=1)
post_season_df.columns = post_season_df.columns.str.lower()

post_season_df = post_season_df.rename(columns = {'off_fg%':'off_fg_percent','off_3p%':'off_3p_percent','off_2p%':'off_2p_percent','off_ft%':'off_ft_percent',
                                     'def_fg%':'def_fg_percent','def_3p%':'def_3p_percent','def_2p%':'def_2p_percent',
                                    'def_ft%':'def_ft_percent','w/l%':'w_l_percent','ts%':'ts_percent','off_efg%':'off_efg_percent',
                                    'off_tov%':'off_tov_percent','off_orb%':'off_orb_percent','off_ft/fga':'off_ft_fga','def_efg%':'def_efg_percent',
                                    'def_tov%':'def_tov_percent','def_drb%':'def_drb_percent','def_ft/fga':'def_ft_fga','3par':'three_par',
                                    'attend./g':'attend_g'})
# post_season_df.columns

### Create Table for Team Names and Abbreviations

In [17]:
teams_df = post_season_df[['team']]
teams_df = teams_df.rename(columns = {'team':'team_name'})
teams_df.head()

,team_name
0,Golden State Warriors
1,Memphis Grizzlies
2,Denver Nuggets
3,New Orleans Pelicans
4,Minnesota Timberwolves


In [18]:
team_names_series = regular_season_df["team"]

team_abbv = ["MIN", "MEM", "MIL", "CHA", "PHX", "ATL", "UTA", "SAS", "BKN", "DEN", "LAL", "BOS", "CHI", "IND", "GSW", "SAC", "MIA", "PHI", "HOU", "TOR", "NOP", "WAS", "LAC", "DAL", "CLE", "NYK", "POR", "DET", "ORL", "OKC"]
team_abbv_series = pd.Series(team_abbv)

frame = { 'team_name': team_names_series, 'abbreviation': team_abbv_series }

name_abbv = pd.DataFrame(frame)

name_abbv

,team_name,abbreviation
0,Minnesota Timberwolves,MIN
1,Memphis Grizzlies,MEM
2,Milwaukee Bucks,MIL
3,Charlotte Hornets,CHA
4,Phoenix Suns,PHX
5,Atlanta Hawks,ATL
6,Utah Jazz,UTA
7,San Antonio Spurs,SAS
8,Brooklyn Nets,BKN
9,Denver Nuggets,DEN


### Process Regular Season Match Data Scraped from NBA Website

In [27]:
df = pd.read_csv('Data/Scraped/NBA_Data_Reg_Season_2022.csv')
df = df.drop(['Unnamed: 0'],axis=1)
df.head()

,team,min,pts,fgm,fga,threepm,threepa,ftm,fta,oreb,dreb,reb,ast,stl,blk,tov,pf,point_diff
0,BOS,240,139,54,99,18,48,13,13,14,42,56,34,5,2,15,20,29
1,CLE,240,133,51,94,19,38,12,17,10,38,48,39,5,5,12,26,18
2,MIL,240,115,39,88,12,30,25,32,8,33,41,27,7,2,12,14,-18
3,ATL,240,130,45,87,21,38,19,24,13,37,50,29,4,4,13,19,16
4,WAS,240,108,42,94,8,23,16,26,17,26,43,25,12,3,11,16,-16


In [28]:
# Determine if game is win or loss
def winloss(row):
    if row['point_diff'] < 0:
        return 'L'
    return 'W'

In [32]:
# Determine if game is away or home
def type(row):
    if row['new'] % 2 == 0:
        return 'vs' #Home
    return '@' # Away

In [35]:
# def team(row):
#     if row['new'] % 2 == 0:
#         return row['new'] - 1
#     return row['new'] + 1

In [65]:
# Add win/loss ('w_l') column
df['w_l'] = df.apply(lambda row: winloss(row), axis=1)

# Move w_l to location that matches other tables
column_to_reorder = df.pop('w_l')
df.insert(1, 'w_l', column_to_reorder)

In [65]:
# Add game type (away or home) column
df['new'] = df.index
df['type'] = df.apply(lambda row: type(row), axis=1)

# Move game type to location that matches other tables
column_to_reorder = df.pop('type')
df.insert(1, 'type', column_to_reorder)

### Process Match Data

In [36]:
regular_matches_df = pd.read_csv('Data/NBA_Data_Reg_Season_2022_plus.csv')
regular_matches_df = regular_matches_df.drop(['Unnamed: 0'],axis=1)
# regular_matches_df.head()

In [64]:
post_matches_df = pd.read_csv('Data/NBA_Data_Playoffs_2022_plus.csv')
post_matches_df = post_matches_df.drop(['Unnamed: 0'],axis=1)
# post_matches_df.head()

# Create Engine and Send Tables to SQL

In [19]:
rds_connection_string = "postgres:postgres@localhost:5432/group_2_project_4"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [20]:
insp = inspect(engine)
insp.get_table_names()

['teams',
 'matches',
 'reg_season_matches',
 'post_season_matches',
 'reg_season_stats',
 'post_season_stats']

In [21]:
name_abbv.to_sql(name='teams', con=engine, if_exists='append', index=False)

In [22]:
pd.read_sql_query('select * from teams', con=engine)

,team_name,abbreviation
0,Minnesota Timberwolves,MIN
1,Memphis Grizzlies,MEM
2,Milwaukee Bucks,MIL
3,Charlotte Hornets,CHA
4,Phoenix Suns,PHX
5,Atlanta Hawks,ATL
6,Utah Jazz,UTA
7,San Antonio Spurs,SAS
8,Brooklyn Nets,BKN
9,Denver Nuggets,DEN


In [23]:
regular_season_df.to_sql(name='reg_season_stats', con=engine, if_exists='append', index=False)

In [24]:
pd.read_sql_query('select * from reg_season_stats', con=engine).head()

,team,mp,off_fg,off_fga,off_fg_percent,off_3p,off_3pa,off_3p_percent,off_2p,off_2pa,...,ts_percent,off_efg_percent,off_tov_percent,off_orb_percent,off_ft_fga,def_efg_percent,def_tov_percent,def_drb_percent,def_ft_fga,attend_g
0,Minnesota Timberwolves,241,42,91,0,15,41,0,27,50,...,1,1,12,24,0,1,14,75,0,16028
1,Memphis Grizzlies,241,44,94,0,12,33,0,32,62,...,1,1,11,30,0,1,13,78,0,15775
2,Milwaukee Bucks,241,42,89,0,14,38,0,28,51,...,1,1,12,23,0,1,12,79,0,17453
3,Charlotte Hornets,242,43,91,0,14,38,0,29,53,...,1,1,12,23,0,1,13,75,0,17092
4,Phoenix Suns,241,44,90,0,12,32,0,32,58,...,1,1,12,22,0,1,13,77,0,16175


In [25]:
post_season_df.to_sql(name='post_season_stats', con=engine, if_exists='append', index=False)

In [26]:
pd.read_sql_query('select * from post_season_stats', con=engine).head()

,team,mp,off_fg,off_fga,off_fg_percent,off_3p,off_3pa,off_3p_percent,off_2p,off_2pa,...,ts_percent,off_efg_percent,off_tov_percent,off_orb_percent,off_ft_fga,def_efg_percent,def_tov_percent,def_drb_percent,def_ft_fga,attend_g
0,Golden State Warriors,240,42,86,0,14,36,0,28,50,...,1,1,13,24,0,1,12,78,0,None
1,Memphis Grizzlies,240,40,91,0,13,36,0,27,56,...,1,1,12,26,0,1,16,78,0,None
2,Denver Nuggets,240,39,83,0,11,31,0,28,51,...,1,1,15,28,0,1,12,81,0,None
3,New Orleans Pelicans,240,39,84,0,9,26,0,30,58,...,1,1,13,35,0,1,11,80,0,None
4,Minnesota Timberwolves,240,36,82,0,14,36,0,23,46,...,1,1,16,17,0,1,14,73,0,None


In [37]:
regular_matches_df.to_sql(name='reg_season_matches', con=engine, if_exists='append', index=False)

In [38]:
pd.read_sql_query('select * from reg_season_matches', con=engine).head()

,team,type,opponent,w_l,min,pts,fgm,fga,threepm,threepa,...,oreb,dreb,reb,ast,stl,blk,tov,pf,point_diff,id
0,MIN,vs,CHI,L,240,120,46,91,11,31,...,9,23,32,30,7,9,13,23,-4,1
1,CHI,@,MIN,W,240,124,44,83,10,21,...,16,32,48,22,9,3,23,22,4,2
2,PHI,vs,DET,W,240,118,46,88,5,25,...,10,32,42,25,13,6,11,23,12,3
3,DET,@,PHI,L,240,106,38,83,11,34,...,15,27,42,26,4,4,20,16,-12,4
4,MEM,vs,BOS,L,240,110,39,102,15,47,...,19,26,45,27,11,6,10,16,-29,5


In [40]:
post_matches_df.to_sql(name='post_season_matches', con=engine, if_exists='append', index=False)

In [41]:
pd.read_sql_query('select * from post_season_matches', con=engine).head()

,team,type,opponent,w_l,min,pts,fgm,fga,threepm,threepa,...,oreb,dreb,reb,ast,stl,blk,tov,pf,point_diff,id
0,GSW,vs,BOS,W,240,107,39,86,15,37,...,6,36,42,25,15,2,12,17,19,1
1,BOS,@,GSW,L,240,88,30,80,15,37,...,6,37,43,24,5,7,19,18,-19,2
2,GSW,vs,BOS,L,240,108,39,88,19,45,...,12,27,39,24,8,6,14,16,-12,3
3,BOS,@,GSW,W,240,120,43,85,21,41,...,7,32,39,33,7,6,13,13,12,4
4,MIA,vs,BOS,L,240,96,37,88,6,30,...,7,37,44,16,4,5,10,18,-4,5
